# CSP Coding Assignment: Sudoku Solver

## Introduction
In this assignment, you will extend the Sudoku-solving agent developed in the classroom lectures to solve diagonal Sudoku puzzles. A diagonal Sudoku puzzle is identical to traditional Sudoku puzzles with the added constraint that the boxes on the two main diagonals of the board must also contain the digits 1-9 in each cell (just like the rows, columns, and 3x3 blocks). You will also implement another strategy called "Naked Pairs", described [here](https://www.learn-sudoku.com/naked-pairs.html)
<img style="float: center;height:350px;" src="naked-twins.png"><br>

In [35]:
def cross(a, b):
      return [s+t for s in a for t in b]

In [36]:
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)

In [37]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

# TODO: creat the diagonal units of the boars
diag1 = [rows[i]+cols[i] for i in range(9)]
diag2 = [rows[i]+cols[8-i] for i in range(9)]

unitlist = row_units + column_units + square_units + [diag1] + [diag2]

In [38]:
# TODO: Update the unit list to add the new diagonal units
unitlist = row_units + column_units + square_units + [diag1] + [diag2]

In [39]:
# Must be called after all units (including diagonals) are added to the unitlist
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [40]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1 + max(len(values[a]) for a in boxes)

    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))

    return

In [41]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = dict(zip(boxes, ['123456789']*81))

    for box, digit in zip(boxes, grid):
        if digit in '123456789':
            values[box] = digit
    return values

In [42]:
def eliminate(values):
    """Apply the eliminate strategy to a Sudoku puzzle

    The eliminate strategy says that if a box has a value assigned, then none
    of the peers of that box can have the same value.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the assigned values eliminated from peers
    """
    
    solved_boxes = [box for box in values.keys() if len(values[box]) == 1]

    for box in solved_boxes:
        digit = values[box]

        for peer in peers[box]:

            values[peer] = values[peer].replace(digit, '')
    return values
    
    


In [43]:
def only_choice(values):
    """Apply the only choice strategy to a Sudoku puzzle

    The only choice strategy says that if only one box in a unit allows a certain
    digit, then that box must be assigned that digit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with all single-valued boxes assigned 
    """

    for unit in unitlist:

        for digit in '123456789':

            boxes_num = [box for box in unit if digit in values[box]]

            if len(boxes_num) == 1:
                values[boxes_num[0]] = digit
    return values


In [44]:
def naked_pairs(values):
    """Eliminate values using the naked pairs strategy.

    The naked pairs strategy says that if you have two or more unallocated boxes
    in a unit and there are only two digits that can go in those two boxes, then
    those two digits can be eliminated from the possible assignments of all other
    boxes in the same unit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked pairs eliminated from peers      
    """
    
    for unit in unitlist:

        unused_boxes = [box for box in unit if len(values[box]) > 1]
 
        pairs = [(box1, box2) for i, box1 in enumerate(unused_boxes)
        
        for box2 in unused_boxes[i+1:]
            if values[box1] == values[box2] and len(values[box1]) == 2]

        for box1, box2 in pairs:
            for box in unit:
                if box != box1 and box != box2:
                    for digit in values[box1]:
                        values[box] = values[box].replace(digit, '')
    return values
    


In [45]:
def reduce_puzzle(values):
    """Reduce a Sudoku puzzle by repeatedly applying all constraint strategies

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary after continued application of the constraint strategies
        no longer produces any changes, or False if the puzzle is unsolvable 
        
    Notes
    -----
    complete this function using elimination and only choice startegies then extending it to call the 
    naked pairs strategy.
    """

    stalled = False
    while not stalled:

        before = len([box for box in values.keys() if len(values[box]) == 1])
        
        values = eliminate(values)
        values = only_choice(values)
        values = naked_pairs(values)
        
        after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = before == after

        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False

    return values

In [46]:
def search(values):
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles
    that cannot be solved by repeated reduction alone.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary with all boxes assigned or False
    """

    values = reduce_puzzle(values)
    
    if values is False:
        return False
    if all(len(values[s]) == 1 for s in boxes):
        return values

    s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)    

    for value in values:
        new_values = values.copy()
        new_values = value
        attempt = search(new_values)
        if attempt:
            return attempt
    

    return False
    



In [47]:
def solve(grid):
    """Find the solution to a Sudoku puzzle using search and constraint propagation

    Parameters
    ----------
    grid(string)
        a string representing a sudoku grid.
        
        Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

    Returns
    -------
    dict or False
        The dictionary representation of the final sudoku grid or False if no solution exists.
    """

    values = grid_values(grid)
    values = search(values)
    return values

In [48]:
def main():
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid_values(diag_sudoku_grid))
    result = solve(diag_sudoku_grid)
    print('\n------------------------------------ Solution ---------------------------------------------\n')
    display(result)

In [49]:
main()

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     3     

------------------------------------ Solution ---------------------------------------------

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
5 7 6 